In [1]:
%pip install opencv-python


   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 435.7 kB/s eta 0:01:30
   ---------------------------------------- 0.0/38.8 MB 326.8 kB/s eta 0:01:59
   ---------------------------------------- 0.1/38.8 MB 731.4 kB/s eta 0:00:53
   ---------------------------------------- 0.3/38.8 MB 1.4 MB/s eta 0:00:29
    --------------------------------------- 0.5/38.8 MB 2.4 MB/s eta 0:00:17
   - -------------------------------------- 1.1/38.8 MB 4.0 MB/s eta 0:00:10
   - -------------------------------------- 1.8/38.8 MB 5.5 MB/s eta 0:00:07
   -- ------------------------------------- 2.6/38.8 MB 6.9 MB/s eta 0:00:06
   --- ------------------------------------ 3.5/38.8 MB 8.2 MB/s eta 0:00:05
   ---- ----------------------------------- 4.3/38.8 MB 9.1 MB/s eta 0:00:04
   ----- ---------------------------------- 5.5/38.8 MB 10.4 MB/s eta 0:00:04
   --


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\rachi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import cv2 as cv
image = cv.imread('1colour.jpg')
cv.imshow('Image', image)
cv.waitKey(0)
cv.destroyAllWindows()

: 